## Analyzing Historical Visitation Numbers of U.S. National Parks
https://github.com/carl-schick-ds/national-parks

***

This project will analyze the historical visitation counts for each U.S. National Park and attempt to answer the following questions: 
 - How does the age of a national park impact annual visitation?
 - How does the area-size of a national park impact annual visitation?
 - Are visitation numbers impacted when a site receives an official national park designation?
 - Does the location of a national park impact annual visitation?
 - How does the weather impact monthly visitation?

***

### Data Collection
The first step of the project is to collect the data from various sources on the internet.  

Data is currently collected from two main sources...
 - The National Park Services' Integrated Resourcde Management Applications (IRMA) Portal
   - List of official national park unit codes
   - Monthly visitation counts for each national park
 - Wikipedia
   - Location (lat/long), date established, and gross area acres for each national park

Web scraping is performed using the [Beautfiul Soup](https://www.crummy.com/software/BeautifulSoup/) library.  
Fuzzy matching logic for park names is performed using the [The Fuzz (formerly Fuzzy Wuzzy)](https://github.com/seatgeek/thefuzz) library.

**_Data collection will rebuild/refresh the CSV files used for the remainder of the notebook._**  
**_Data collection can be toggled on/off with the REFRESH_DATA boolean flag._**

In [31]:
REFRESH_DATA = False

import numpy as np
import pandas as pd
import os

if REFRESH_DATA:
    # pip install thefuzz

    from bs4 import BeautifulSoup
    from thefuzz import fuzz
    from thefuzz import process
    import requests
    import time
    import re
    user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    headers = {'User-Agent': user_agent}

In [32]:
if REFRESH_DATA:
    from requests.adapters import HTTPAdapter
    from requests.packages.urllib3.util.retry import Retry
    retry_strategy = Retry(
        total=3,
        backoff_factor=1,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["HEAD", "GET", "OPTIONS"]
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    http = requests.Session()
    http.mount("https://", adapter)
    http.mount("http://", adapter)

In [33]:
if REFRESH_DATA:

    park_units_url = 'https://irmaservices.nps.gov/v2/rest/unit/designations'
    park_units_namespace = {'root': 'NRPC.IrmaServices.Rest.Unit'}
    park_unit_exceptions = {'DENG':'DENA', 'GAAG':'GAAR', 'GLBG':'GLBA', 'GRDG':'GRSA', 'KATG':'KATM', 'LACG':'LACL', 'WRSG':'WRST'}
    
    national_parks = {}
    r = http.get(park_units_url, headers=headers)
    soup = BeautifulSoup(r.text, "xml")
    # print(soup)

    for unit_designation in soup.find_all('UnitDesignation'):
        if unit_designation.find("Code").text == 'NP':
            units = unit_designation.find("Units")
            for value in units.find_all('Value'):
                raw_code = value.find("Code").text
                name = value.find("Name").text
                code = raw_code if raw_code not in park_unit_exceptions.keys() else park_unit_exceptions[raw_code]
                national_parks[code] = name

    # Manually add New River Gorge, if it hasn't been added yet
    if 'NERI' not in national_parks.keys():
        national_parks['NERI'] = 'New River Gorge'

    # print(national_parks)
    park_units_df = pd.DataFrame.from_dict(national_parks, orient='index', columns=['name'])
    park_units_df.index.name = 'code'
    park_units_df.sort_index(inplace=True)
    park_units_df.to_csv('national_park_units.csv')
    # print(park_units_df.head())

In [34]:
if REFRESH_DATA:

    # park_visits_homepage = 'https://irma.nps.gov/STATS/'
    park_visits_domain = 'https://irma.nps.gov'
    park_visits_url = '/STATS/SSRSReports/Park%20Specific%20Reports/Recreation%20Visitors%20By%20Month%20(1979%20-%20Last%20Calendar%20Year)'
    park_visits_qs = '?Park='
    park_visits_df = pd.DataFrame()
    target_table_min = 10

    print('Processing:', end=" ")
    for park_code in park_units_df.index:
        print(park_code, end=", ")
        park_visits_request = park_visits_domain + park_visits_url + park_visits_qs + park_code
        r = http.get(park_visits_request, headers=headers, timeout=5)
        soup = BeautifulSoup(r.text, "html")
        park_visits_iframe = soup.find('iframe').attrs['src']

        park_visits_request = park_visits_domain + park_visits_iframe
        r = http.get(park_visits_request, headers=headers, timeout=5)

        dfs = pd.read_html(r.text, match="Year", skiprows=1)
        for df in dfs:
            if len(df) > target_table_min: one_park_df = df
        
        new_header = one_park_df.iloc[0] #grab the first row for the header
        one_park_df = one_park_df[1:] #take the data less the header row
        one_park_df.columns = new_header #set the header row as the df header
    
        one_park_df = one_park_df.fillna(0)
        if 'Total' in one_park_df.columns:
            one_park_df.drop('Total', axis=1, inplace=True)

        one_park_df.set_index('Year', inplace=True)
        one_park_srs = one_park_df.stack()

        park_visits_df[park_code] = one_park_srs

    park_visits_df = park_visits_df.fillna(0)
    park_visits_df.index.names = ['Year', 'Month']
    park_visits_df.to_csv('national_park_visits.csv')


In [35]:
if REFRESH_DATA:

    park_data_url = 'https://en.wikipedia.org/wiki/List_of_national_parks_of_the_United_States'
    
    r = http.get(park_data_url, headers=headers)
    soup = BeautifulSoup(r.text, "xml")
    # print(soup)

    dfs = pd.read_html(r.text, match="Date established as park")
    park_data_df = dfs[0]

    # Strip years and footnotes from columns
    for column_name in park_data_df.columns.values:
        if '(' in column_name or '[' in column_name:
            new_column_name = re.sub("\(.*?\)","", column_name)
            new_column_name = re.sub("\[.*?\]","", new_column_name)
            new_column_name = new_column_name.strip()
            park_data_df.rename(columns={column_name: new_column_name}, inplace=True)
 
    # Rename some columns and drop unneeded columns
    park_data_df.rename(columns={'Date established as park': 'Established', 'Area': 'Acres'}, inplace=True)
    park_data_df.drop(columns=['Image', 'Recreation visitors'], inplace=True)

    # Remove asterisks from the park names
    park_data_df['Name'] = park_data_df['Name'].str.replace('*', '').str.strip()

    # Add each park's unit code
    # We will need fuzzy string matching logic to match the name from Wikipedia with the name from NPS
    nps_names = park_units_df['name'].to_list()
    # print(nps_names)
    for index, row in park_data_df.iterrows():
        # print(park_data_df.loc[index,'Name'])
        matching_name, matching_ratio = process.extractOne(park_data_df.loc[index,'Name'], nps_names)
        # print(matching_name, matching_ratio)
        # print(park_units_df[park_units_df['name'] == matching_name].index.tolist()[0])
        park_data_df.loc[index,'Code'] = park_units_df[park_units_df['name'] == matching_name].index.tolist()[0]
    
    # Parse state from Location into new column; update Location to only lat/long coordinates
    park_data_df['State'] = park_data_df['Location'].apply(lambda x: re.split('[^a-zA-Z\s\.]', x)[0].replace('.mw', ''))
    park_data_df['Location'] = park_data_df['Location'].apply(lambda x: x.rpartition('/')[2].strip())

    # Remove footnotes from established dates
    park_data_df['Established'] = park_data_df['Established'].apply(lambda x: re.sub('\[\d*?\]', '', x).strip())

    # Clean Acres column
    park_data_df['Acres'] = park_data_df['Acres'].apply(lambda x: re.split('[^\d\,\.]', x)[0])

    park_data_df = park_data_df[['Code', 'Name', 'State', 'Location', 'Established', 'Acres', 'Description']]

    park_data_df.set_index('Code', inplace=True)
    park_data_df.to_csv('national_park_data.csv')
    # print(park_data_df.head())

### Data Loading and Cleaning

In [36]:
# Read in the CSV files
park_visits_df = pd.read_csv('national_park_visits.csv', index_col=[0,1])
park_data_df = pd.read_csv('national_park_data.csv', index_col=0)
# print(park_visits_df.head())
# print(park_data_df.head())

In [37]:
# Quick review of Park Visits data
print(park_visits_df.info())
print(park_visits_df.describe())
print(park_visits_df.isna().sum())

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 516 entries, (2021, 'JAN') to (1979, 'DEC')
Data columns (total 63 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   ACAD    516 non-null    int64
 1   ARCH    516 non-null    int64
 2   BADL    516 non-null    int64
 3   BIBE    516 non-null    int64
 4   BISC    516 non-null    int64
 5   BLCA    516 non-null    int64
 6   BRCA    516 non-null    int64
 7   CANY    516 non-null    int64
 8   CARE    516 non-null    int64
 9   CAVE    516 non-null    int64
 10  CHIS    516 non-null    int64
 11  CONG    516 non-null    int64
 12  CRLA    516 non-null    int64
 13  CUVA    516 non-null    int64
 14  DENA    516 non-null    int64
 15  DEVA    516 non-null    int64
 16  DRTO    516 non-null    int64
 17  EVER    516 non-null    int64
 18  GAAR    516 non-null    int64
 19  GLAC    516 non-null    int64
 20  GLBA    516 non-null    int64
 21  GRBA    516 non-null    int64
 22  GRCA    516 non-null    int6

In [38]:
# Quick review of Park Data data
print(park_data_df.info())
print(park_data_df.isna().sum())

<class 'pandas.core.frame.DataFrame'>
Index: 63 entries, ACAD to ZION
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Name         63 non-null     object
 1   State        63 non-null     object
 2   Location     63 non-null     object
 3   Established  63 non-null     object
 4   Acres        63 non-null     object
 5   Description  63 non-null     object
dtypes: object(6)
memory usage: 3.4+ KB
None
Name           0
State          0
Location       0
Established    0
Acres          0
Description    0
dtype: int64


In [39]:
# Clean up Park Data by converting Established to datetime and Acres to float
park_data_df['Established'] = pd.to_datetime(park_data_df['Established'])
park_data_df['Acres'] = park_data_df['Acres'].str.replace(',', '').astype(float)
park_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63 entries, ACAD to ZION
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Name         63 non-null     object        
 1   State        63 non-null     object        
 2   Location     63 non-null     object        
 3   Established  63 non-null     datetime64[ns]
 4   Acres        63 non-null     float64       
 5   Description  63 non-null     object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 3.4+ KB


In [40]:
# Sum monthly visits into annual totals and merge the visit totals into the park-data dataframe
park_annual_visits_df = park_visits_df.groupby(level=0).sum().transpose()
park_data_df = park_data_df.join(park_annual_visits_df)
park_data_df.head()

,Name,State,Location,Established,Acres,Description,1979,1980,1981,1982,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Code,,,,,,,,,,,,,,,,,,,,,
ACAD,Acadia,Maine,﻿44.35°N 68.21°W,1919-02-26,49076.63,Covering most of Mount Desert Island and other...,2787366,2779666,2997972,3572114,...,2431052,2254922,2563129,2811184,3303393,3509271,3537575,3437286,2669034,4054767
NPSA,American Samoa,American Samoa,﻿14.25°S 170.68°W,1988-10-31,8256.67,The southernmost national park is on three Sam...,0,0,0,0,...,10440,17919,13953,13892,28892,69468,28626,60006,4819,7284
ARCH,Arches,Utah,﻿38.68°N 109.57°W,1971-11-12,76678.98,"This site features more than 2,000 natural san...",269840,290519,326508,339415,...,1070577,1082866,1284767,1399247,1585718,1539028,1663557,1659702,1238083,1750341
BADL,Badlands,South Dakota,﻿43.75°N 102.50°W,1978-11-10,242755.94,"The Badlands are a collection of buttes, pinna...",858000,952652,1175952,1030484,...,883406,892372,868094,989354,996263,1054325,1008942,970998,916932,1228481
BIBE,Big Bend,Texas,﻿29.25°N 103.25°W,1944-06-12,801163.21,Named for the prominent bend in the Rio Grande...,282941,174008,167332,180144,...,292055,316953,314102,381747,388290,440276,440091,463832,393907,524126


In [46]:
# Let's calculate the first year of a Park as their established year, if they were established betwen Jan-June; or the following year if they were established Jul-Dec.
park_data_df['First Year'] = park_data_df['Established'].apply(lambda x: x.year if x.month <= 6 else x.year + 1)

# Since we only have visit data from 1979 and beyond, we will filter out parks established prior to 1981.  
# That will give us at least 2 years of visit data before the park became a National Park.
park_data_filtered_df = park_data_df[park_data_df['First Year'] > 1980]
print(park_data_filtered_df.shape)
print(park_data_filtered_df[['Name', 'Established', 'First Year']])

(63, 50)
                              Name Established  First Year
Code                                                      
NPSA                American Samoa  1988-10-31        1989
BLCA  Black Canyon of the Gunnison  1999-10-21        2000
CONG                      Congaree  2003-11-10        2004
CUVA               Cuyahoga Valley  2000-10-11        2001
DEVA                  Death Valley  1994-10-31        1995
DRTO                  Dry Tortugas  1992-10-26        1993
GAAR           Gates of the Arctic  1980-12-02        1981
JEFF                  Gateway Arch  2018-02-22        2018
GLBA                   Glacier Bay  1980-12-02        1981
GRBA                   Great Basin  1986-10-27        1987
GRSA              Great Sand Dunes  2004-09-24        2005
INDU                 Indiana Dunes  2019-02-15        2019
JOTR                   Joshua Tree  1994-10-31        1995
KATM                        Katmai  1980-12-02        1981
KEFJ                  Kenai Fjords  1980-12-02 